In [ ]:
!pip install geopandas
import geopandas
import pandas as pd
!pip install contextily
import contextily as ctx

In [ ]:
import os
!pip install rtree
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
os.chdir("/content/spatialindex-src-1.8.5")
!./configure
!make
!make install
!ldconfig

Caricare i geodataframe dei comuni, delle province e delle regioni ed eliminare tutte le colonne ad eccezione di quelle che contengono il nome e la geometria. Utilizzare poi le join spaziali per visualizzare:


In [ ]:
comuni = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Limiti01012021_g/Com01012021_g/Com01012021_g_WGS84.dbf")
province = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Limiti01012021_g/ProvCM01012021_g/ProvCM01012021_g_WGS84.dbf")
regioni = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/Limiti01012021_g/Reg01012021_g/Reg01012021_g_WGS84.dbf")

In [ ]:
comuni = comuni[["COMUNE", "geometry"]]
province = province[["DEN_UTS", "geometry"]]
regioni = regioni[["DEN_REG", "geometry"]]

1. l'elenco dei comuni con le province in cui si trovano


In [ ]:
com_prov = geopandas.sjoin(comuni, province, op='within', how='left')
com_prov

2. l'elenco delle province con le regioni in cui si trovano


In [ ]:
prov_reg = geopandas.sjoin(province, regioni, op="within", how="left")
prov_reg

3. l'elenco dei comuni con le regioni in cui si trovano


In [ ]:
com_reg = geopandas.sjoin(comuni, regioni, op="within", how="left")
com_reg

4. visualizzare una mappa con le province colorate in base al numero di comuni che hanno


In [ ]:
prov_com = geopandas.sjoin(province, comuni, op="contains", how="left")
num_com_prov = prov_com.groupby("DEN_UTS")["COMUNE"].count().reset_index()
num2 = pd.merge(province, num_com_prov, on = "DEN_UTS", how="left")

ax = num2.to_crs(epsg = 3857).plot(figsize = (20,20), edgecolor = "k", column = "COMUNE", legend = True, alpha = 0.5)
ctx.add_basemap(ax)

5. visualizzare una mappa con le regioni colorate in base al numero di province che hanno


In [ ]:
reg_prov = geopandas.sjoin(regioni, province, op="contains", how="left")
num_reg_prov = reg_prov.groupby("DEN_REG")["DEN_UTS"].count().reset_index()
num3 = pd.merge(regioni, num_reg_prov, on = "DEN_REG", how="left")

ax = num3.to_crs(epsg = 3857).plot(figsize = (20,20), edgecolor = "k", column = "DEN_UTS", legend = True, alpha = 0.5)
ctx.add_basemap(ax)